# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: xx

Group Members:
- Name (NIM)
- Name (NIM)
- ...

## Import Libraries

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from urllib.parse import urlparse, parse_qs
import re
import ipaddress

# Import other libraries if needed

## Import Dataset

In [14]:
# Import PhiUSIIL Phishing URL Dataset
df = pd.read_csv('https://drive.google.com/uc?id=1QZu9z5SMP4u0h0Rs8lVp5u8JJsZ5VvK_') # Import The Dataset Given by The Assistants (From The Spesification Docs)
df_original = df.copy()

# Modify Dataset (Because filename is a feature that's shouldn't be included --> From the metadata of the Dataset)
df = df.drop('FILENAME', axis=1)

# Number of rows and columns
print(f"Dataset shape: {df.shape}")

# Column names and their data types
print("\nColumn info:")
print(df.info())

# Class distribution (legitimate vs phishing)
print("\nClass distribution:")
print(df['label'].value_counts(normalize=True))

# Print The First 5 Rows
df.head()

Dataset shape: (140404, 55)

Column info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140404 entries, 0 to 140403
Data columns (total 55 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          140404 non-null  int64  
 1   URL                         96917 non-null   object 
 2   URLLength                   79765 non-null   float64
 3   Domain                      70207 non-null   object 
 4   DomainLength                94085 non-null   float64
 5   IsDomainIP                  98274 non-null   float64
 6   TLD                         95005 non-null   object 
 7   CharContinuationRate        92362 non-null   float64
 8   TLDLegitimateProb           87531 non-null   float64
 9   URLCharProb                 88333 non-null   float64
 10  TLDLength                   92673 non-null   float64
 11  NoOfSubDomain               96344 non-null   float64
 12  HasObfuscation              74

,id,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,1,https://www.northcm.ac.th,24.0,www.northcm.ac.th,17.0,0.0,NaN,0.800000,NaN,NaN,...,0.0,0.0,1.0,NaN,3.0,NaN,69.0,NaN,NaN,1
1,4,http://uqr.to/1il1z,NaN,NaN,NaN,NaN,to,1.000000,0.000896,0.036850,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0
2,5,https://www.woolworthsrewards.com.au,35.0,www.woolworthsrewards.com.au,28.0,0.0,au,0.857143,NaN,0.060894,...,1.0,0.0,1.0,33.0,7.0,8.0,15.0,NaN,2.0,1
3,6,NaN,31.0,NaN,NaN,NaN,com,0.562500,0.522907,0.055829,...,1.0,0.0,1.0,24.0,5.0,14.0,NaN,NaN,NaN,1
4,11,NaN,NaN,www.nyprowrestling.com,22.0,0.0,NaN,1.000000,NaN,NaN,...,0.0,0.0,1.0,NaN,NaN,14.0,NaN,0.0,NaN,1


In [15]:
def correct_data_types(df):
    # Create a dataframe copy to avoid modifying the original
    df_correct = df.copy()

    # Binary features to convert to boolean
    binary_features = [
        'HasTitle', 'HasFavicon', 'HasDescription', 'HasSocialNet',
        'HasSubmitButton', 'HasHiddenFields', 'HasPasswordField',
        'HasCopyrightInfo', 'HasExternalFormSubmit', 'IsDomainIP',
        'IsHTTPS', 'IsResponsive', 'HasObfuscation', 'Bank', 'Pay',
        'Crypto', 'Robots', 'label'
    ]

    # Text features to convert to string
    text_features = ['URL', 'Domain', 'Title']

    # Convert binary features to boolean
    for col in binary_features:
        df_correct[col] = df_correct[col].astype("boolean")
    
    # Convert text features to string
    for col in text_features:
        if col in df_correct.columns:
            df_correct[col] = df_correct[col].astype("string")

    return df_correct

In [16]:
# Apply the function
df_correct = correct_data_types(df)

# Number of rows and columns
print(f"Dataset shape: {df_correct.shape}")

# Column names and their data types
print("\nColumn info:")
print(df_correct.info())

# Class distribution (legitimate vs phishing)
print("\nClass distribution:")
print(df_correct['label'].value_counts(normalize=True))

# Global variables for numerical columns and categorical columns (reusability purposes)
numerical_columns = df_correct.select_dtypes(include=['int64', 'float64']).columns 
categorical_columns = df_correct.select_dtypes(include=['object', 'boolean']).columns
text_columns = df_correct.select_dtypes(include=['string']).columns

# Drop ID from Numerical Columns
numerical_columns = numerical_columns.drop('id')

# Global variables for target columns (The Output Variable)
target_column = 'label'

df_correct.head()

Dataset shape: (140404, 55)

Column info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140404 entries, 0 to 140403
Data columns (total 55 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          140404 non-null  int64  
 1   URL                         96917 non-null   string 
 2   URLLength                   79765 non-null   float64
 3   Domain                      70207 non-null   string 
 4   DomainLength                94085 non-null   float64
 5   IsDomainIP                  98274 non-null   boolean
 6   TLD                         95005 non-null   object 
 7   CharContinuationRate        92362 non-null   float64
 8   TLDLegitimateProb           87531 non-null   float64
 9   URLCharProb                 88333 non-null   float64
 10  TLDLength                   92673 non-null   float64
 11  NoOfSubDomain               96344 non-null   float64
 12  HasObfuscation              74

,id,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,1,https://www.northcm.ac.th,24.0,www.northcm.ac.th,17.0,False,NaN,0.800000,NaN,NaN,...,False,False,True,NaN,3.0,NaN,69.0,NaN,NaN,True
1,4,http://uqr.to/1il1z,NaN,<NA>,NaN,<NA>,to,1.000000,0.000896,0.036850,...,<NA>,False,False,NaN,NaN,NaN,NaN,NaN,1.0,False
2,5,https://www.woolworthsrewards.com.au,35.0,www.woolworthsrewards.com.au,28.0,False,au,0.857143,NaN,0.060894,...,True,False,True,33.0,7.0,8.0,15.0,NaN,2.0,True
3,6,<NA>,31.0,<NA>,NaN,<NA>,com,0.562500,0.522907,0.055829,...,True,False,True,24.0,5.0,14.0,NaN,NaN,NaN,True
4,11,<NA>,NaN,www.nyprowrestling.com,22.0,False,NaN,1.000000,NaN,NaN,...,False,False,True,NaN,NaN,14.0,NaN,0.0,NaN,True


# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [17]:
def create_train_val_split(df, target_col='label', test_size=0.2, random_state=42, verbose=True):
    """
    Creates training and validation splits while preserving class distributions.
    
    Args:
    df: DataFrame containing the complete dataset
    target_col: Name of the target column (default: 'label')
    test_size: Proportion of dataset to include in the validation split (default: 0.2)
    random_state: Random state for reproducibility (default: 42)
    verbose: Whether to print split statistics (default: True)
    
    Returns:
    tuple: (train_data, validation_data)
    """
    
    # Separate features and target
    X = df.drop(target_col, axis=1)
    y = df[target_col]
    
    # Create stratified split
    X_train, X_val, y_train, y_val = train_test_split(
        X, y,
        test_size=test_size,
        random_state=random_state,
        stratify=y
    )
    
    # Reconstruct complete dataframes with all features including target
    train_data = X_train.copy()
    train_data[target_col] = y_train
    
    validation_data = X_val.copy()
    validation_data[target_col] = y_val
    
    if verbose:
        print("Dataset Split Summary")
        print("-" * 50)
        print(f"\nTotal samples: {len(df)}")
        print(f"Training set size: {len(train_data)} ({len(train_data)/len(df)*100:.2f}%)")
        print(f"Validation set size: {len(validation_data)} ({len(validation_data)/len(df)*100:.2f}%)")
        
        # Class distribution in original dataset
        print("\nOriginal class distribution:")
        orig_dist = df[target_col].value_counts(normalize=True)
        print(f"Legitimate: {orig_dist[True]*100:.2f}%")
        print(f"Phishing: {orig_dist[False]*100:.2f}%")
        
        # Class distribution in training set
        print("\nTraining set class distribution:")
        train_dist = train_data[target_col].value_counts(normalize=True)
        print(f"Legitimate: {train_dist[True]*100:.2f}%")
        print(f"Phishing: {train_dist[False]*100:.2f}%")
        
        # Class distribution in validation set
        print("\nValidation set class distribution:")
        val_dist = validation_data[target_col].value_counts(normalize=True)
        print(f"Legitimate: {val_dist[True]*100:.2f}%")
        print(f"Phishing: {val_dist[False]*100:.2f}%")
        
        # Feature completeness check
        print("\nFeature completeness in training set:")
        train_nulls = train_data.isnull().sum() / len(train_data) * 100
        print(train_nulls.sort_values(ascending=False))
        
        print("\nFeature completeness in validation set:")
        val_nulls = validation_data.isnull().sum() / len(validation_data) * 100
        print(val_nulls.sort_values(ascending=False))
    
    return train_data, validation_data

train_data, validation_data = create_train_val_split(df_correct)

def save_split_datasets(train_data, validation_data, train_path='train_data.csv', val_path='validation_data.csv'):
    """
    Saves the training and validation sets to CSV files
    
    Args:
    train_data: Training DataFrame
    validation_data: Validation DataFrame
    train_path: Path to save training data
    val_path: Path to save validation data
    """
    train_data.to_csv(train_path, index=False)
    validation_data.to_csv(val_path, index=False)
    print(f"\nDatasets saved to {train_path} and {val_path}")

save_split_datasets(train_data, validation_data)

Dataset Split Summary
--------------------------------------------------

Total samples: 140404
Training set size: 112323 (80.00%)
Validation set size: 28081 (20.00%)

Original class distribution:
Legitimate: 92.48%
Phishing: 7.52%

Training set class distribution:
Legitimate: 92.48%
Phishing: 7.52%

Validation set class distribution:
Legitimate: 92.48%
Phishing: 7.52%

Feature completeness in training set:
Domain                        49.964834
NoOfExternalRef               49.454698
LineOfCode                    49.250821
LargestLineLength             48.408607
HasSocialNet                  48.398814
NoOfURLRedirect               48.008867
HasCopyrightInfo              47.967914
NoOfCSS                       47.760476
NoOfObfuscatedChar            47.579748
NoOfSelfRedirect              47.561052
HasPasswordField              47.282391
LetterRatioInURL              46.890664
HasObfuscation                46.739314
ObfuscationRatio              45.832999
NoOfLettersInURL             

# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [18]:
def split_dataset_by_url(df, verbose=True):
    """
    Splits the dataset into two parts based on URL availability
    
    Args:
    df: Original DataFrame
    verbose: Whether to print summary statistics
    
    Returns:
    tuple: (complete_url_data, missing_url_data)
    """
    # Create the splits
    complete_url_data = df[df['URL'].notna()].copy()
    missing_url_data = df[df['URL'].isna()].copy()
    
    if verbose:
        print("Dataset Split Summary")
        print("-" * 50)
        
        # Overall statistics
        print(f"\nOriginal dataset size: {len(df):,} rows")
        print(f"Complete data size: {len(complete_url_data):,} rows ({len(complete_url_data)/len(df)*100:.2f}%)")
        print(f"Missing URL data size: {len(missing_url_data):,} rows ({len(missing_url_data)/len(df)*100:.2f}%)")
        
        # Class distribution in complete data
        complete_dist = complete_url_data['label'].value_counts(normalize=True)
        print("\nClass distribution in complete data:")
        print(f"Legitimate: {complete_dist[True]*100:.2f}%")
        print(f"Phishing: {complete_dist[False]*100:.2f}%")
        
        # Class distribution in missing URL data
        missing_dist = missing_url_data['label'].value_counts(normalize=True)
        print("\nClass distribution in missing URL data:")
        print(f"Legitimate: {missing_dist[True]*100:.2f}%")
        print(f"Phishing: {missing_dist[False]*100:.2f}%")
        
        # Feature completeness in both datasets
        print("\nFeature completeness in complete data:")
        complete_nulls = complete_url_data.isnull().sum() / len(complete_url_data) * 100
        print(complete_nulls.sort_values(ascending=False))
        
        print("\nFeature completeness in missing URL data:")
        missing_nulls = missing_url_data.isnull().sum() / len(missing_url_data) * 100
        print(missing_nulls.sort_values(ascending=False))
    
    return complete_url_data, missing_url_data

complete_url_data, missing_url_data = split_dataset_by_url(train_data)

def save_split_datasets(complete_url_data, missing_url_data, complete_path='complete_url_data.csv', missing_path='missing_url_data.csv'):
    """
    Saves the split datasets to CSV files
    
    Args:
    complete_url_data: DataFrame with complete URL data
    missing_url_data: DataFrame with missing URL data
    complete_path: Path to save complete data
    missing_path: Path to save missing URL data
    """
    complete_url_data.to_csv(complete_path, index=False)
    missing_url_data.to_csv(missing_path, index=False)
    print(f"\nDatasets saved to {complete_path} and {missing_path}")

save_split_datasets(complete_url_data, missing_url_data)

Dataset Split Summary
--------------------------------------------------

Original dataset size: 112,323 rows
Complete data size: 77,628 rows (69.11%)
Missing URL data size: 34,695 rows (30.89%)

Class distribution in complete data:
Legitimate: 92.53%
Phishing: 7.47%

Class distribution in missing URL data:
Legitimate: 92.38%
Phishing: 7.62%

Feature completeness in complete data:
Domain                        49.953625
NoOfExternalRef               49.466687
LineOfCode                    49.248982
HasSocialNet                  48.389756
LargestLineLength             48.361416
HasCopyrightInfo              47.866749
NoOfURLRedirect               47.844850
NoOfCSS                       47.651621
NoOfObfuscatedChar            47.641315
NoOfSelfRedirect              47.565311
HasPasswordField              47.265162
HasObfuscation                46.850363
LetterRatioInURL              46.792394
ObfuscationRatio              45.685835
NoOfLettersInURL              45.121606
SpacialCharRatio

In [19]:
class URLFeatureExtractor:
    def __init__(self):
        # Regex for IP address detection
        self.ip_pattern = r'^(?:[0-9]{1,3}\.){3}[0-9]{1,3}$'
        
    def extract_features(self, url):
        """Extract all features from a single URL"""
        if pd.isna(url):
            return {}
            
        try:
            # Parse URL
            parsed = urlparse(url)
            domain = parsed.netloc
            
            # Basic features
            features = {
                'URLLength': len(url),
                'Domain': domain,
                'DomainLength': len(domain),
                'TLD': self._get_tld(domain),
                'TLDLength': len(self._get_tld(domain)),
                'NoOfSubDomain': self._count_subdomains(domain),
                'IsHTTPS': url.startswith('https://'),
                'IsDomainIP': self._is_ip_address(domain)
            }
            
            # Character counts and ratios
            char_features = self._analyze_characters(url)
            features.update(char_features)
            
            # Calculate character continuation rate
            features['CharContinuationRate'] = self._calc_char_continuation(url)
            
            return features
            
        except Exception as e:
            print(f"Error processing URL {url}: {str(e)}")
            return {}
    
    def _get_tld(self, domain):
        """Extract TLD from domain"""
        if not domain:
            return ''
        parts = domain.split('.')
        return parts[-1] if len(parts) > 0 else ''
    
    def _count_subdomains(self, domain):
        """Count number of subdomains"""
        if not domain:
            return 0
        # Subtract 1 from dot count (as dots = subdomains + 1)
        return domain.count('.') if domain.count('.') > 0 else 0
    
    def _is_ip_address(self, domain):
        """Check if domain is an IP address"""
        try:
            # Try to create IP address object
            ipaddress.ip_address(domain)
            return True
        except:
            # Check against regex pattern
            return bool(re.match(self.ip_pattern, domain))
    
    def _analyze_characters(self, url):
        """Analyze character types and their ratios in URL"""
        total_len = len(url)
        if total_len == 0:
            return {}
            
        letters = sum(c.isalpha() for c in url)
        digits = sum(c.isdigit() for c in url)
        equals = url.count('=')
        qmarks = url.count('?')
        ampersands = url.count('&')
        
        # Count other special characters
        special_chars = sum(not c.isalnum() for c in url) - equals - qmarks - ampersands
        
        return {
            'NoOfLettersInURL': letters,
            'LetterRatioInURL': letters / total_len,
            'NoOfDegitsInURL': digits,
            'DegitRatioInURL': digits / total_len,
            'NoOfEqualsInURL': equals,
            'NoOfQMarkInURL': qmarks,
            'NoOfAmpersandInURL': ampersands,
            'NoOfOtherSpecialCharsInURL': special_chars,
            'SpacialCharRatioInURL': (special_chars + equals + qmarks + ampersands) / total_len
        }
    
    def _calc_char_continuation(self, url):
        """Calculate character continuation rate"""
        if not url:
            return 0
            
        # Find longest sequences of same type (letter/digit/special)
        def get_type(c):
            if c.isalpha(): return 'alpha'
            if c.isdigit(): return 'digit'
            return 'special'
            
        current_type = get_type(url[0])
        current_length = 1
        max_lengths = {'alpha': 1, 'digit': 1, 'special': 1}
        
        for c in url[1:]:
            c_type = get_type(c)
            if c_type == current_type:
                current_length += 1
                max_lengths[c_type] = max(max_lengths[c_type], current_length)
            else:
                current_type = c_type
                current_length = 1
        
        # Sum of longest sequences divided by URL length
        total_continuation = sum(max_lengths.values())
        return total_continuation / len(url)

def fill_url_derived_features(df):
    """Fill all URL-derived features in the DataFrame"""
    # Create copy to avoid modifying original
    df_filled = df.copy()
    
    # Initialize extractor
    extractor = URLFeatureExtractor()
    
    # Get URLs that need processing (have at least one missing derived feature)
    derived_features = [
        'URLLength', 'Domain', 'DomainLength', 'TLD', 'TLDLength',
        'NoOfSubDomain', 'IsHTTPS', 'IsDomainIP', 'NoOfLettersInURL',
        'LetterRatioInURL', 'NoOfDegitsInURL', 'DegitRatioInURL',
        'NoOfEqualsInURL', 'NoOfQMarkInURL', 'NoOfAmpersandInURL',
        'NoOfOtherSpecialCharsInURL', 'SpacialCharRatioInURL',
        'CharContinuationRate'
    ]
    
    # Process each URL with missing features
    for idx, row in df_filled.iterrows():
        if row['URL'] is not None and any(pd.isna(row[feat]) for feat in derived_features):
            features = extractor.extract_features(row['URL'])
            for feat, value in features.items():
                if pd.isna(df_filled.at[idx, feat]):
                    df_filled.at[idx, feat] = value
    
    return df_filled

df_filled = fill_url_derived_features(complete_url_data)

def analyze_derived_features(df_original, df_filled):
    """Analyze the results of feature derivation"""
    print("Feature Filling Analysis:")
    print("-" * 50)
    
    for col in df_filled.columns:
        orig_missing = df_original[col].isna().sum()
        filled_missing = df_filled[col].isna().sum()
        if orig_missing != filled_missing:
            print(f"\n{col}:")
            print(f"Original missing values: {orig_missing}")
            print(f"Remaining missing values: {filled_missing}")
            print(f"Filled values: {orig_missing - filled_missing}")

analyze_derived_features(complete_url_data, df_filled)

def save_url_derived_datasets(derived_data, derived_path='derived_url_data.csv'):
    """
    Saves the split datasets to CSV files
    
    Args:
    complete_data: DataFrame with complete URL data
    missing_url_data: DataFrame with missing URL data
    complete_path: Path to save complete data
    missing_path: Path to save missing URL data
    """
    derived_data.to_csv(derived_path, index=False)
    print(f"\nDatasets saved to {derived_path}")

save_url_derived_datasets(df_filled)

Feature Filling Analysis:
--------------------------------------------------

URLLength:
Original missing values: 33545
Remaining missing values: 0
Filled values: 33545

Domain:
Original missing values: 38778
Remaining missing values: 0
Filled values: 38778

DomainLength:
Original missing values: 25649
Remaining missing values: 0
Filled values: 25649

IsDomainIP:
Original missing values: 23335
Remaining missing values: 0
Filled values: 23335

TLD:
Original missing values: 25115
Remaining missing values: 0
Filled values: 25115

CharContinuationRate:
Original missing values: 26563
Remaining missing values: 0
Filled values: 26563

TLDLength:
Original missing values: 26347
Remaining missing values: 0
Filled values: 26347

NoOfSubDomain:
Original missing values: 24254
Remaining missing values: 0
Filled values: 24254

NoOfLettersInURL:
Original missing values: 35027
Remaining missing values: 0
Filled values: 35027

LetterRatioInURL:
Original missing values: 36324
Remaining missing values: 0


### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [20]:
# Write your code here

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [21]:
# Write your code here

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [22]:
# Write your code here

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [23]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [24]:
# Write your code here

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [25]:
# Write your code here

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [26]:
# Write your code here

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [27]:
# from sklearn.pipeline import Pipeline

# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [28]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [29]:
# Write your code here

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [30]:
# Type your code here

## B. Naive Bayes

In [31]:
# Type your code here

## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [32]:
# Type your code here

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [33]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`